# Code Written by:
**Shweta Tiwari**
*20 Oct 2023*

## Algorithm: Hashtable Open Addressing

In [ ]:
import time

In [ ]:
import numpy as np

# Algorithm

In [ ]:
%%time
class HashTable:

    ratio_expand = .7
    ratio_shrink = .2
    min_size = 11
    empty = (None,)

    def __init__(self, size=None):
        self._size = size or self.min_size
        self._buckets = [None] * self._size
        self._count = 0

    def _entry(self, key):
        # get hash
        hash_ = hash(key)
        idx1 = None

        for i in range(self._size):
            # quadratic probing
            idx = (hash_ + i) % self._size
            entry = self._buckets[idx]

            # end of chain
            if not entry:
                break
            # remember first empty bucket
            elif entry is self.empty:
                if idx1 is None:
                    idx1 = idx
            # test key
            elif entry[0] == key:
                return idx, entry

        else:
            # out of space
            if idx1 is None:
                raise IndexError()

        # return first empty bucket
        return (idx, None) if idx1 is None else (idx1, None)

    def _ensure_capacity(self):
        fill = self._count / self._size

        # expand or shrink?
        if fill > self.ratio_expand:
            self._size = self._size * 2 + 1
        elif fill < self.ratio_shrink and self._size > self.min_size:
            self._size = (self._size - 1) // 2
        else:
            return

        # reallocate buckets
        entries = self._buckets
        self._buckets = [None] * self._size

        # store entries into new buckets
        for entry in entries:
            if entry and entry is not self.empty:
                idx, _ = self._entry(entry[0])
                self._buckets[idx] = entry

    def __len__(self):
        return self._count

    def __contains__(self, key):
        _, entry = self._entry(key)
        return bool(entry)

    def __getitem__(self, key):
        _, entry = self._entry(key)
        return entry and entry[1]

    def __setitem__(self, key, value):
        idx, entry = self._entry(key)

        # set value
        self._buckets[idx] = key, value

        # expand
        self._count += bool(not entry or entry is self.empty)
        self._ensure_capacity()

    def __delitem__(self, key):
        idx, entry = self._entry(key)

        # delete key and value
        if entry:
            self._buckets[idx] = self.empty

        # shrink
        self._count -= bool(entry and entry is not self.empty)
        self._ensure_capacity()

    def __iter__(self):
        for entry in self._buckets:
            if entry and entry is not self.empty:
                yield entry[0]

    def slots(self):
        return ''.join('-' if not p else 'o' if p is self.empty else 'x' for p in self._buckets)

CPU times: user 45 µs, sys: 0 ns, total: 45 µs
Wall time: 50.1 µs


# Run

In [ ]:
%%time
table = HashTable()

CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 14.1 µs


In [ ]:
%%time
# add random values
for _ in range(1000):
    key, value = np.random.randint(1000), np.random.rand()
    if np.random.rand() >= .5:
        table[key] = value
    else:
        del table[key]

CPU times: user 14.1 ms, sys: 0 ns, total: 14.1 ms
Wall time: 20 ms


In [ ]:
%%time
len(table), table._size

CPU times: user 7 µs, sys: 1 µs, total: 8 µs
Wall time: 11.4 µs


(303, 767)

In [ ]:
%%time
table.slots()

CPU times: user 142 µs, sys: 20 µs, total: 162 µs
Wall time: 166 µs


'-x----xxx--o--xx--xxxxox-o-xx-x----o-xx----xxxxx-x--xxx-xxx-xxxxxxxxx--x--xxo-o---x--xxxo-xxxxxxxxxx--xxxo-xxxxxxx-x-xxxxxxxxxxxxxxxxoxxxxxxxox-oxoxo-xxxxx-xx--xxxxxxxxxxxxxxxxxxxxxxxx---x--x-xx-xx-xxoxxxxxxxxxxx-xxx---x-----x--x-x-x-o--x-ox--x--xx--xx----ox-xxx---x-x---ox------x---x--x-o---x-xoxxxxx--oxx---x-----x-xxxxx--x-o---x-x-x-----o--ox---xx-x-x----------o-xx-xx-x-x--xx-x-xx--xxx---x---o-o----------o--o-----xx--o-xx-----------x-o-----x-----x--x-xo----x---x-o--------x--x-----x-------xx--x--x-x---xx-o-----xxx-----x-x-----x--x--x-xx----------x-------x----x--------------x--x--x-------x--x---ox-x--xx-xo-----xx-xxxx------x--xxx-x--xx----x---x--x-x-x-x--x-x--x---o-x------x--xx-x----xx----x---x-x-x--x--------x-xx--xx-oxoxxx----------x-xx--x-x--xx---x-xo--x--'

In [ ]:
%%time
# print some values
for key in list(table)[:5]:
    print(key, table[key])

1 0.9291936460819349
6 0.5201999127450451
773 0.8342257734660241
774 0.1927176298523572
14 0.3190177035790148
CPU times: user 1.92 ms, sys: 7 µs, total: 1.93 ms
Wall time: 3.04 ms


In [ ]:
%%time
# delete all the values
for key in list(table):
    del table[key]

CPU times: user 1.43 ms, sys: 60 µs, total: 1.49 ms
Wall time: 1.5 ms


In [ ]:
%%time
len(table), table._size

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 9.3 µs


(0, 11)

# The End